In [3]:
# -*- coding: utf-8 -*- 
# Reference : https://github.com/jbhoosreddy/spellcorrect
# 4 Confusion matrix (addition, substituion, reversal, deletion) files
# English corpus file
# English dictionary file
# Errors of spell file
# Above things are referred to that github repository as written.

# This program is to correct non-word spelling error in sentences using ngram MAP language models,
# noisy channel model, error confusion matrix and Damerau-Levenshtein edit distance.
# Usage --
# Input : she is a briliant acress
# Response : She is a brilliant actress

from __future__ import division
from ngram import nGram 
import ast                       # To read confusion matrix files
import hgtk
import math as calc              # To calculate mathematical forms
import locale
import functools
import re
import pandas as pd
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Komoran
komoran = Komoran()
from pykospacing import spacing
from konlpy.tag import Hannanum
hannanum = Hannanum()
from konlpy.tag import Twitter
twitter = Twitter()
# Class name : SpellCorrect
import preprocessing
from konlpy.corpus import kolaw

In [4]:
class NoisyChannelModel():

    # Method : __init__(self)
    # A constructor to declare ngram language model, load words, confusion matrix and dictionary.
    def __init__(self):
        locale.setlocale(locale.LC_ALL, '') #한국어 기준으로 set

        # Construct nGram Language model for 1-gram, bi-gram 
        self.ng = nGram(uni = True,bi = True)
  #      self.ng_word =  self.open_pre_ngram()

        # During constructing nGram language model,
        # it turn textwords corpus into "set" data structure
        # For instance, set("Hello") --> {'e', 'H', 'l', 'o'}
        # As same, it makes the words corpus as set data structure and sort those things.
        self.words = sorted(set(self.ng.words),key=functools.cmp_to_key(locale.strcoll))
        #ng_word
        # Read 4 confusion matrix external text file and assign it to its ~~~matrix variables.
        self.loadConfusionMatrix()

        # Read English dictionaly external text file and split it into word by enter character '\n'
        self.dict = self.loadDict()

        # End method
        return
    
    """def open_pre_ngram(self):
        f =open('pre_ngram_words.txt','r')
        word = f.read().split('\n')
        return word"""
    
    # Method : loadConfusionMatrix(self)
    # A method to load confusion matrix from external confusion matrix data file
    def loadConfusionMatrix(self):
        # Read addition confusion matrix from external text file
        #df = pd.read_csv('subtestmatrix.txt',encoding='cp949', names=['x','y','count'])
        self.addmatrix = pd.read_csv('addtestmatrix.txt',encoding='cp949', names=['x','y','count'])

        # Read substitution confusion matrix from external text file
        self.submatrix = pd.read_csv('subtestmatrix.txt',encoding='cp949', names=['x','y','count'])

        # Read reverse confusion matrix from external text file
        self.revmatrix = pd.read_csv('revtestmatrix.txt',encoding='cp949', names=['x','y','count'])

        # Read deletion confusion matrix from external text file
        self.delmatrix = pd.read_csv('deltestmatrix.txt',encoding='cp949', names=['x','y','count'])

    # Method : loadDict(self)
    # A method to load dictionary file in directory into memory
    def loadDict(self):
        #print("Loading dictionary from data file")
        f = open('dictionary.txt','r') # ,encoding='cp949'원래 dictionary.txt
        #f = open('semi_dictionary.txt','r',encoding='utf-8') # Test용 'ㅇ' 부분만 자른 semi_ictionary.txt
        return f.read().split("\n")
    
        
    # Method : editDistanceOfDamerauLevenshtein(self, s1, s2)
    # A method to calculate Damerau-Levenshtein which expands minimum edit distance method for given two strings.
    def editDistanceOfDamerauLevenshtein(self, source, target):  #내가 입력한 단어, 후보 검사할 단어
        # Concatenate '#' to two strings
        # Get each length of two strings
        source = '#' + source
        target = '#' + target
        #print('source : ', source)
        #print('target : ', target)
        source = re.sub('ᴥ','',source)
        target = re.sub('ᴥ','',target)
        lengthOfSource = len(source) #내가 입력한 단어 자소분리 후 길이 저장
        lengthOfTarget = len(target) #후보 검사할 단어 자소분리 후 길이 저장
        #print('lengthOfSource : ', lengthOfSource)
        #print('lengthOfTarget : ', lengthOfTarget)
        
        # Calculate Damerau-Levenshtein edit distance matrix in distanceMatrix
        distanceMatrix = [[0]*lengthOfTarget for i in range(lengthOfSource)]
        for i in range(lengthOfSource):
             for j in range(lengthOfTarget):
                    distanceMatrix[i][0] = i
                    distanceMatrix[0][j] = j
                 
        for i in range(lengthOfSource):
            for j in range(lengthOfTarget):
                # distance variable to calculate distance between source and target string
                distance = [0] * 4
                if i == 0 or j == 0:
                    continue
                # Assign the values in distance matrix
                distance[0] = distanceMatrix[i-1][j] + 1
                distance[1] = distanceMatrix[i][j-1] + 1
                # Select the minimum value for making chart of minimum edit distance according to Damerau-Levenshtein edit distance algorithm
                if source[i] != target[j]:
                    distance[2] = distanceMatrix[i-1][j-1] + 2
                else:
                    distance[2] = distanceMatrix[i-1][j-1]
                if source[i] == target[j-1] and source[i-1] == target[j]:
                    distance[3] = distanceMatrix[i-1][j-1] - 1
                if distance[3] != 0:
                    distanceMatrix[i][j] = min(distance[0:4])
                else:
                    distanceMatrix[i][j] = min(distance[0:3])
                    
        # Return distance value matrix between source and target strings
        return distanceMatrix[lengthOfSource-1][lengthOfTarget-1]
    
    def splitSentence(self, sentence):
        komoran = Komoran()
        sentence = komoran.morphs(sentence)
        return sentence
        
    def splitWord(self, word):
        word = re.sub('ㅙ','ㅗㅐ',word)
        word = re.sub('ㅘ','ㅗㅏ',word)
        word = re.sub('ㅚ','ㅗㅣ',word)
        word = re.sub('ㅞ','ㅜㅔ',word)
        word = re.sub('ㅟ','ㅜㅣ',word)
        word = re.sub('ㅞ','ㅜㅔ',word)
        word = re.sub('ㅝ','ㅜㅓ',word)
        word = re.sub('ㅢ','ㅡㅣ',word)
        word = re.sub('ㅖ','ㅕㅣ',word)
        word = re.sub('ㅒ','ㅑㅣ',word)
        word = re.sub('ㄲ','ㄱㄱ',word)
        word = re.sub('ㄸ','ㄷㄷ',word)
        word = re.sub('ㅃ','ㅂㅂ',word)
        word = re.sub('ㅆ','ㅅㅅ',word)
        word = re.sub('ㅉ','ㅈㅈ',word)
        word = re.sub('ㄳ','ㄱㅅ',word)
        word = re.sub('ㄵ','ㄴㅈ',word)
        word = re.sub('ㄶ','ㄴㅎ',word)
        word = re.sub('ㄺ','ㄹㄱ',word)
        word = re.sub('ㄻ','ㄹㅁ',word)
        word = re.sub('ㄼ','ㄹㅂ',word)
        word = re.sub('ㄽ','ㄹㅅ',word)
        word = re.sub('ㄾ','ㄹㅌ',word)
        word = re.sub('ㄿ','ㄹㅍ',word)
        word = re.sub('ㅀ','ㄹㅎ',word)
        word = re.sub('ㅄ','ㅂㅅ',word)
        return word
 
    def uniteWord(self, word):
        word = re.sub('ㅗㅐ','ㅙ',word)
        word = re.sub('ㅗㅏ','ㅘ',word)
        word = re.sub('ㅗㅣ','ㅚ',word)
        word = re.sub('ㅜㅔ','ㅞ',word)
        word = re.sub('ㅜㅣ','ㅟ',word)
        word = re.sub('ㅜㅔ','ㅞ',word)
        word = re.sub('ㅜㅓ','ㅝ',word)
        word = re.sub('ㅡㅣ','ㅢ',word)
        word = re.sub('ㅕㅣ','ㅖ',word)
        word = re.sub('ㅑㅣ','ㅒ',word)
        word = re.sub('ㄱㄱ','ㄲ',word)
        word = re.sub('ㄷㄷ','ㄸ',word)
        word = re.sub('ㅂㅂ','ㅃ',word)
        word = re.sub('ㅅㅅ','ㅆ',word)
        word = re.sub('ㅈㅈ','ㅉ',word)
        word = re.sub('ㄱㅅ','ㄳ',word)
        word = re.sub('ㄴㅈ','ㄵ',word)
        word = re.sub('ㄴㅎ','ㄶ',word)
        word = re.sub('ㄹㄱ','ㄺ',word)
        word = re.sub('ㄹㅁ','ㄻ',word)
        word = re.sub('ㄹㅂ','ㄼ',word)
        word = re.sub('ㄹㅅ','ㄽ',word)
        word = re.sub('ㄹㅌ','ㄾ',word)
        word = re.sub('ㄹㅍ','ㄿ',word)
        word = re.sub('ㄹㅎ','ㅀ',word)
        word = re.sub('ㅂㅅ','ㅄ',word)
        return word
    
    def extractCan(self, word):
        # addConfusionMatrix # x가 xy로 쓰여진 경우 / x가 candidate letter, xy가 input letter
        candidates = [word]
        candidates.append("ADD : ")
        for i in range(len(word)-1): # 입력 자소 차례로 받아오기
            if i == 0:
                candidate = word[i+1:]
                candidates.append(candidate)
            elif i < len(word) - 1:
                candidate = word[:i]+word[i+1:] # y부분 제거
                candidates.append(candidate)
                if candidate[i-1] == 'ᴥ' and candidate[i] == 'ᴥ': # 모음이 추가된 경우
                    candidate = word[:i-1]+word[i+2:] 
                    candidates.append(candidate)
                
        # subConfusionMatrix # x가 y로 쓰여진 경우 / y : input , x : candidate letter
        candidates.append("SUB : ")
        for i in range(len(word)): # 입력 자소 차례로 받아오기
            if word[i] == 'ᴥ':
                continue
            candidates_letter = self.submatrix[(self.submatrix['y']==word[i])]
            candidates_letter = candidates_letter.sort_values(['count'], ascending=[False])
            candidates_letter = candidates_letter[0:15]
            candidates_letter = candidates_letter['x'].values
            # range 수정
            for j in range(10):
                if i == 0:
                    candidate = candidates_letter[j]+word[i+1:]
                    candidates.append(candidate)
                elif i < len(word)-1:
                    candidate = word[:i]+candidates_letter[j]+word[i+1:]
                    candidates.append(candidate)
            
        # revConfusionMatrix # xy가 yx로 쓰여진 경우 / yx는 input xy는 candidate letter
        candidates.append("TRANS : ") # input에서 word[i] = y, word[i+1] = x
        for i in range(len(word)-2): #팩토리얼처럼 모든 trans 경우 구함
            if i == 0:
                candidate = word[i+1]+word[i]+word[i+2:] # ex: 괏님(관심)처럼 형태가 정확한 경우
                candidates.append(candidate)
                if i < len(word)-3 and word[i+1] == 'ᴥ':
                    candidate = word[i+2]+word[i+1]+word[i]+word[i+3:] # ex: 괏님(관심)처럼 형태가 정확한 경우
                    candidates.append(candidate)
                if i < len(word)-3 and word[i+1] == 'ᴥ' and word[i+3] == 'ᴥ': #자,모가 바뀌어 ᴥ 모양이 있는 경우
                    if i < len(word)-5 and word[i+5] == 'ᴥ': #받침있는 
                        candidate = word[i+2]+word[i]+word[i+4:] #자+모(ex:ㅓ,ㅅ,ㅇ,공)
                        candidates.append(candidate)
                        candidate = word[i]+word[i+4]+word[i+2]+word[i+5:] #모+자(ex:ㅅ,ㅇ,ㅓ,공)
                        candidates.append(candidate)
                    else: #받침없는 자+모(ex:ㅏ,ㅁ,술), 자+자(받침, ex: 싷,ㄹ,다)
                        candidate = word[i+2]+word[i]+word[i+3:]
                        candidates.append(candidate)
                elif i < len(word)-4 and word[i+1] == 'ᴥ' and word[i+4] == 'ᴥ': #모+자(ex: ㅅ,어,공)
                    candidate = word[i]+word[i+3]+word[i+2]+word[i+4:]
                    candidates.append(candidate)
                        
            else:
                candidate = word[:i]+word[i+1]+word[i]+word[i+2:] # ex: 괏님(관심)처럼 형태가 정확한 경우
                candidates.append(candidate)
                if i < len(word)-3 and word[i+1] == 'ᴥ':
                    candidate = word[:i]+word[i+2]+word[i+1]+word[i]+word[i+3:] # ex: 괏님(관심)처럼 형태가 정확한 경우
                    candidates.append(candidate)
                if i < len(word)-3 and word[i+1] == 'ᴥ' and word[i+3] == 'ᴥ': #자,모가 바뀌어 ᴥ 모양이 있는 경우
                    if i < len(word)-5 and word[i+5] == 'ᴥ': #받침있는 
                        candidate = word[:i]+word[i+2]+word[i]+word[i+4:] #자+모(ex:ㅓ,ㅅ,ㅇ,공)
                        candidates.append(candidate)
                        candidate = word[:i+1]+word[i+4]+word[i+2]+word[i+5:] #모+자(ex:ㅅ,ㅇ,ㅓ,공)
                        candidates.append(candidate)
                    else: #받침없는 자+모(ex:ㅏ,ㅁ,술), 자+자(받침, ex: 싷,ㄹ,다)
                        candidate = word[:i]+word[i+2]+word[i]+word[i+3:]
                        candidates.append(candidate)
                elif i < len(word)-4 and word[i+1] == 'ᴥ' and word[i+4] == 'ᴥ': #모+자(ex: ㅅ,어,공)
                    candidate = word[:i+1]+word[i+3]+word[i+2]+word[i+4:]
                    candidates.append(candidate)
                    
        # delConfusionMatrix # xy가 x로 쓰여진 경우 / xy가 candidate letter, x가 input letter
        candidates.append("DEL : ")
        for i in range(len(word)-1): # 입력 자소 차례로 받아오기
            if word[i] == 'ᴥ':
                continue
            candidates_letter = self.delmatrix[(self.delmatrix['x']==word[i])]
            candidates_letter = candidates_letter.sort_values(['count'], ascending=[False])
            candidates_letter = candidates_letter[0:15]
            candidates_letter = candidates_letter['y'].values
            for j in range(10):
                if i == 0:
                    if i < len(word)-3 and word[i+1] == 'ᴥ' and word[i+3] == 'ᴥ': # 받침있는 단어의 
                        candidate = candidates_letter[j]+word[i]+word[i+2:] #초성이 없는 경우(ex: ㅏ,ㄴ,녕), candidates_letter[j]는 중성과 연관
                        candidates.append(candidate)
                        candidate = word[i]+candidates_letter[j]+word[i+2:] #중성이 없는 경우(ex: ㅇ,ㄴ,녕), candidates_letter[j]는 초성과 연관
                        candidates.append(candidate)
                    else: 
                        candidate = word[i:i+2]+candidates_letter[j]+word[i+2:] # 받침있는 단어의 종성이 없는 경우(ex: 아,녕)
                        candidates.append(candidate)
                        candidate = candidates_letter[j]+word[i:] # 받침없는 단어의 초성이 없는 경우
                        candidates.append(candidate)
                        candidate = word[i]+candidates_letter[j]+word[i+1:]  # 받침없는 단어의 중성이 없는 경우
                        candidates.append(candidate)
                else:
                    if i > 1 and i < len(word)-3 and word[i+1] == 'ᴥ' and word[i+3] == 'ᴥ': # 받침있는 단어(초성,중성)
                        # 앞글자가 받침이 있을 때 현재 단어의 초성이 없는 경우(ex: 콘,센,트->코,넨,트), candidates_letter[j]는 중성과 연관
                        candidate = word[:i-1]+word[i]+word[i-1]+candidates_letter[j]+word[i+1]
                        candidates.append(candidate)
                        # 앞글자가 받침이 없을 때 현재 단어의 초성이 없는 경우(ex: 커,튼->커,ㅡ,ㄴ), candidates_letter[j]는 중성과 연관
                        candidate = word[:i]+candidates_letter[j]+word[i]+word[i+2:] 
                        candidates.append(candidate)
                        
                        # 앞글자가 받침이 없을 때 현재 단어의 중성이 없는 경우(ex: 전,자,석->전,잣,ㄱ), candidates_letter[j]는 초성과 연관
                        candidate = word[:i]+word[i+1]+word[i]+candidates_letter[j]+word[i+2:]
                        candidates.append(candidate)
                        # 앞글자가 받침이 있을 때 현재 단어의 중성이 없는 경우(ex: 콘,센,트->콘,ㅔ,ㄴ,트), candidates_letter[j]는 초성과 연관
                        candidate = word[:i]+word[i]+candidates_letter[j]+word[i+2:] 
                        candidates.append(candidate)
                    else: # 받침있는 단어(종성) + 받침없는 단어(초성,중성)
                        # 받침있는 단어의 종성이 없는 경우(ex: 아,녕) (앞글자 상관없음)
                        candidate = word[:i+2]+candidates_letter[j]+word[i+2:] 
                        candidates.append(candidate)
                        
                        # 앞글자가 받침이 있을 때 현재 단어의 초성이 없는 경우(ex: 핸,드,폰->해,느,폰)
                        candidate = word[:i-1]+word[i]+word[i-1]+candidates_letter[j]+word[i+1:] 
                        candidates.append(candidate)
                        # 앞글자가 받침이 없을 때 현재 단어의 초성이 없는 경우(ex: 모,기,약->모,ㅣ,약)
                        candidate = word[:i]+candidates_letter[j]+word[i:] 
                        candidates.append(candidate)
                        # 위와 동일하지만 특수한 경우(ex: 고,기,압->괴압)
                        candidate = word[:i-2]+word[i-1]+candidates_letter[j]+word[i-2]+word[i-1:] 
                        candidates.append(candidate)
                        
                        # 앞글자가 받침이 없을 때 현재 단어의 중성이 없는 경우(ex: 모,기,약->목,약)
                        candidate = word[:i-2]+word[i-1]+word[i-2]+candidates_letter[j]+word[i-1:]  
                        candidates.append(candidate)
                        # 앞글자가 받침이 있을 때 현재 단어의 중성이 없는 경우(ex: 핸,드,폰->핸,ㄷ,폰)
                        candidate = word[:i+1]+candidates_letter[j]+word[i+1:]  
                        candidates.append(candidate)
                    
        #print(candidates)
        return candidates

    # Method : getCandidates(self, word)
    # A method to generate set of candidates for a givne word using Damerau-Levenshtein edit distance
    can_dict = list()
    def genCandidates(self, word): # 자소 분리한 입력 단어를 매개변수로 받음
        # Construct dictionary data structure
        global can_dict
        can_dict = self.extractCan(word)
      #  print('can_dict로 후보 추출한 결과 : ', can_dict)
        candidates = dict()
        
        for item in can_dict:
            compose_item = self.uniteWord(item)
            compose_item = hgtk.text.compose(compose_item)
            if compose_item in self.dict:
                # Along for loop, it calculate Damerau-Levenshtein edit distance between word and item with respect to all words corpus
                distance = self.editDistanceOfDamerauLevenshtein(word, item)
                # If Damerau-Levenshtein edit distance is less than 1,
                # it assigns distance calculated above to candidates dictionary as key-value(item, distance).
                #if item[-1]=='*'  and distance <= 2:
                #    print(item[-1])
                #    candidates[item] = distance
         #       print('후보 : ', item, '거리 : ', distance)
                if distance <= 1:
                    candidates[item] = distance
       #             print('candidate:',item)
                

        # Return sorted dictionary data structure
        return sorted(candidates, key = candidates.get, reverse = False)

    # Method : editType(self, candidate, word)
    # A method to calculate edit type for single edit errors
    # It can be one of four type, addition, deletion, substitution, reverse
    def editType(self, candidate, word):
        # The variable for classfying type of confusion matrix. Now it has default value, false.
        edit = [False] * 4
        correct = ""
        error = ""
        x = ''
        w = ''
        word = re.sub('ᴥ','',word)
        candidate = re.sub('ᴥ','',candidate) #원활한 error, correct 계산을 위해 자소 분리에서 생성된 'ᴥ' 임시 제거
        # Classifying the type of edit by Damerau-Levenshtein edit distance
        # edit[0] = Insertion
        # edit[1] = Deletion
        # edit[2] = Substitution
        # edit[3] = Transposition
        minimum = min([len(word),len(candidate)]) 
        if minimum == 1 :
        #for i in range(minimum):
        # del :
             i = 0
             if len(candidate) > len(word):
                if candidate[i:] == word[i-1:]: #Deletion
                    edit[1] = True
                    correct = candidate[i-1]
                    error = ''
                    x = candidate[i-2]
                    w = candidate[i-2] + candidate[i-1]
                  #  break
                
                elif len(candidate) < len(word): #Insertion 
                    correct = ''
                    error = word[i]
                    if i == 0:
                        w = '#'
                        x = '#' + error
                    else:
                        w = word[i-1]
                        x = word[i-1] + error
                        edit[0] = True
                    #    break
                
                else :
                    if candidate[i+1:] == word[i+1:]: #Substitution
                        edit[2] = True
                        correct = candidate[i]
                        error = word[i]
                        x = error
                        w = correct
                 #       break
                
                    elif candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]: #Transposition
                        edit[3] = True
                        correct = candidate[i] + candidate[i+1]
                        error = word[i] + word[i+1]
                        x = error
                        w = correct
                    #    break
        else :
            for i in range(min([len(word),len(candidate)])-1):
                #print('word: ', word, 'candidate: ', candidate)
                if candidate[0:i+1] != word[0:i+1]:
                    if candidate[i:] == word[i-1:]: #Deletion
                        edit[1] = True
                        correct = candidate[i-1]
                        error = ''
                        x = candidate[i-2]
                        w = candidate[i-2] + candidate[i-1]
                        break

                    elif candidate[i:] == word[i+1:]: #Insertion 
                        correct = ''
                        error = word[i]
                        if i == 0:
                            w = '#'
                            x = '#' + error
                        else:
                            w = word[i-1]
                            x = word[i-1] + error
                            edit[0] = True
                            break

                    if candidate[i+1:] == word[i+1:]: #Substitution
                        edit[2] = True
                        correct = candidate[i]
                        error = word[i]
                        x = error
                        w = correct
                        break

                    if candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]: #Transposition
                        edit[3] = True
                        correct = candidate[i] + candidate[i+1]
                        error = word[i] + word[i+1]
                        x = error
                        w = correct
                        break

        # Reverses both matrix
        candidate = candidate[::-1]
        word = word[::-1]

        minimum = min([len(word),len(candidate)]) 
        if minimum == 1 :
        #for i in range(minimum):
        # del :
            i = 1
            if len(candidate) > len(word):
                if candidate[i:] == word[i-1:]: #Deletion
                    edit[1] = True
                    correct = candidate[i-1]
                    error = ''
                    x = candidate[i-2]
                    w = candidate[i-2] + candidate[i-1]
                #    break

                elif len(candidate) < len(word): #Insertion 
                    correct = ''
                    error = word[i]
                    if i == 0:
                        w = '#'
                        x = '#' + error
                    else:
                        w = word[i-1]
                        x = word[i-1] + error
                        edit[0] = True
                  #      break

                else :
                    if candidate[i+1:] == word[i+1:]: #Substitution
                        edit[2] = True
                        correct = candidate[i]
                        error = word[i]
                        x = error
                        w = correct
                     #   break

                    elif candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]: #Transposition
                        edit[3] = True
                        correct = candidate[i] + candidate[i+1]
                        error = word[i] + word[i+1]
                        x = error
                        w = correct
                    #    break
        # Classifying the type of edit by Damerau-Levenshtein edit distance once again
        else :
            for i in range(min([len(word),len(candidate)])-1):
                if candidate[0:i+1] != word[0:i+1]:

                    if candidate[i:] == word[i-1:]:
                        edit[1] = True
                        correct = candidate[i-1]
                        error = ''
                        x = candidate[i-2]
                        w = candidate[i-2] + candidate[i-1]
                        break

                    elif candidate[i:] == word[i+1:]:    
                        correct = ''
                        error = word[i]
                        if i == 0:
                            w = '#'
                            x = '#'+error

                        else:
                            w = word[i-1]
                            x = word[i-1] + error
                            edit[0] = True
                            break

                    if candidate[i+1:] == word[i+1:]:
                        edit[2] = True
                        correct = candidate[i]
                        error = word[i]
                        x = error
                        w = correct
                        break

                    if candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]:
                        edit[3] = True
                        correct = candidate[i] + candidate[i+1]
                        error = word[i] + word[i+1]
                        x = error
                        w = correct
                        break
        # Returns the variables including edit types, correct ,error, x, w
        # Example :
        # Error | Correction | Correct Letter | Error Letter | Position(Letter #) | Type
        # acress   actress           t              ----            2               deletion
        # acress    cress          ----              a              0               insertion
        # acress    caress          ca              ac              0             transposition
        # acress    access          c               r               2               substitution
        # acress    across          o               e               3               substitution
        # acress    acres          ----             s               5               insertion
        # acress    acres          ----             s               4               insertion
   #     print("word : ", word, "candidate : ", candidate)
        if word == candidate:
            return "None", '', '', '', ''
        
        if edit[1]:
            return "Deletion", correct, error, x, w
        
        elif edit[0]:
            return "Insertion", correct, error, x, w
        
        elif edit[2]:
            return "Substitution", correct, error, x, w
        
        elif edit[3]:
            return "Reversal", correct, error, x, w

    # Method : channelMode(self, x, y, edit)
    # A method to calculate channel model probability for errors
    def channelModel(self, x,y, edit):
        corpus = ' '.join(self.words)   #ng_word
        
        #print("x:",x,"y:",y,"edit:",edit)
        
        if edit == 'add':
            if x == '#':
       #         print("return:",self.addmatrix[(self.addmatrix['x']==x)&(self.addmatrix['y']==y)]['count']/(corpus.count(' '+y)+0.1))
                return (self.addmatrix[(self.addmatrix['x']==x)&(self.addmatrix['y']==y)]['count']/(corpus.count(' '+y)+0.1)).values[0]
            else:
         #       print("return:",self.addmatrix[(self.addmatrix['x']==x)&(self.addmatrix['y']==y)]['count']/(corpus.count(x)+0.1))
                return (self.addmatrix[(self.addmatrix['x']==x)&(self.addmatrix['y']==y)]['count']/(corpus.count(x)+0.1)).values[0]
            
        if edit == 'sub':
        #    print("return:",self.submatrix[(self.submatrix['x']==x)&(self.submatrix['y']==y)]['count']/(corpus.count(y)+0.1))
            return (self.submatrix[(self.submatrix['x']==x)&(self.submatrix['y']==y)]['count']/(corpus.count(y)+0.1)).values[0]
        
        if edit == 'rev':
       #     print("return:",self.revmatrix[(self.revmatrix['x']==x)&(self.revmatrix['y']==y)]['count']/(corpus.count(x+y)+0.1))
            return (self.revmatrix[(self.revmatrix['x']==x)&(self.revmatrix['y']==y)]['count']/(corpus.count(x+y)+0.1)).values[0]
        
        if edit == 'del':
      #      print("return:",self.delmatrix[(self.delmatrix['x']==x)&(self.delmatrix['y']==y)]['count']/(corpus.count(x+y)+0.1))
            return (self.delmatrix[(self.delmatrix['x']==x)&(self.delmatrix['y']==y)]['count']/(corpus.count(x+y)+0.1)).values[0]
        
######################################################################
# section below starts main function to correct non-word spell errors.
#
# Overall Program Flow :
# 1. Input the distorted sentence from user
# 2. Generates the candidates which is answer corresponding to input
# 3. Calculates edit distance by Damerau-Levenshtein edit distance algorithm
# 4. Clasifying the edit type of candidates
# 5. Calculates the sentence probability by selected #gram for each candidate
# 6. Extracts the highest probability as correct answer
# 7. Ends program
######################################################################

# Display the functions of NoisyChannelModel class
# help(NoisyChannelModel)
# Construct a instance of NoisyChannelModel class
josa_word = list()    # 조사 확정 후 저장할 곳 

def detach(word):

    global josa_word
    word_pos_list = list()    # 후보들
    word_select = "" #원래 단어에서 조사 삭제하고 다시 리턴할 단어
    word_pos_list = komoran.pos(word)
    leng = len(josa_word)
   # print("word_pos_list : ",word_pos_list)
    for i in range(len(word_pos_list)):
  #      print("twit : ",twitter.pos(word))
        if 'J' in word_pos_list[i][1] :
            josa_word.append(word_pos_list[i][0]) # 조사 저장
            word_select=re.sub(word_pos_list[i][0],'',word) # 조사 삭제한 단어 저장
        elif twitter.pos(word_pos_list[i][1]) == 'KoreanParticle' :
            #print("particle") #글자내에 ㅏ,ㅓ 같은 자모가 있으면 조사라고 인식하지 못함. ->맨끝 1단어 2단어 검색후 조사면 떼어냄.
            if i != len(word)-1:
                if 'J' in twitter.pos(word[-1])[1] : #맨끝 한글자가 조사
                    josa_word.append(word[-1])
                    word_select = word[:-1]
                if 'J' in twitter.pos(word[-2:])[1]  : 
                    if 'J' in twitter.pos(word[-1])[1] : #맨끝한글자가 조사인데 두글자도 조사
                        josa_word[-1] = word[-2:]   #있던거 없애고 추가하기
                        word_select = word[:-2]
                    else :#맨끝한글자가 조사가 아닌데 두글자는 조사
                        josa_word.append(word[-2:])
                        word_select = word[:-2]
            else :  #조사 오타인 경우
                    word_select = word[:-1]
                    josa_word.append(word[-1])
        else :   # particle이 없음.
            word_select = word
    if len(josa_word) == leng: #단어에 새로운 조사가 없으면
        josa_word.append('')
   # print("word_select : ",word_select)
    if word_select == '' :
        word_select = josa_word[-1]
        josa_word[-1] = ''
    return  word_select


noisyChannelModel = NoisyChannelModel()
Sample = preprocessing.preprocess('test1').prepro()
#print(Sample)
count = 0
while (count != len(Sample)):
    # 입력한 문장의 띄어쓰기 모두 삭제
    start = Sample[count] # str(input('Input : '))    
    count = count+1
    sentence = ""
    for item in start :
        if item == " ":
            continue
        else :
            sentence = sentence + item
            
    sentence = spacing(sentence)   # 올바른 띄어쓰기된 문장
    sentence = sentence.split()   # 단어별로 분리 후 sentence 저장
    print(sentence)
    # 조사 제거
    
    words = list()
    for word in sentence :   #for word in sentence[:-1]
        detach_word = detach(word)
        words.append(detach_word)
        
  #  words.append(sentence[-1])   # 마지막 단어(동사)하나 추가
    sentence = words
    #print("조사 제거 후 : ", sentence , josa_word)
    # User can input a sentence which has been "distorted"
    # sentence = str(input('Input: ')).split()        # 각 단어별로 쪼개서 sentence에 저장
    # sentence = splitSentence(sentence)
    
    correct = ""

    # Starts Noise channel model algorithm
    for index, word in enumerate(sentence):    #enumerate : (index,단어)로 변환
        print("현재 검색 단어 :", word)
        word_input = word
        word = hgtk.text.decompose(word)       #word : 단어를 자소로 분리
        word = noisyChannelModel.splitWord(word)  # splitword : 자소분리된 것 중 ㄲ,ㅒ등 분리
        candidates = noisyChannelModel.genCandidates(word)   # 후보생성
        
        # Constructs two dictionary
        NP = dict()
        P = dict()

        for item in candidates:
            # Gets edit type of each candidate
            edit = noisyChannelModel.editType(item, word)
            
            #print("edit:",edit)
            item = noisyChannelModel.uniteWord(item)
            item = hgtk.text.compose(item)        # 밑에 넣어주려고 임시 결합
            
            # This section classify the edit type by Damerau-Levenshtein edit distance    # 거리 계산
            if edit == None:  
                continue
            
            if edit[0] == "Insertion":
                NP[item] = noisyChannelModel.channelModel(edit[3][0],edit[3][1], 'add')
                
            if edit[0] == 'Deletion':
                NP[item] = noisyChannelModel.channelModel(edit[4][0], edit[4][1], 'del')
                
            if edit[0] == 'Reversal':
                NP[item] = noisyChannelModel.channelModel(edit[4][0], edit[4][1], 'rev')
                
            if edit[0] == 'Substitution':
                NP[item] = noisyChannelModel.channelModel(edit[3], edit[4], 'sub')

        #print("Noise : ", NP)
        # Calculates probability with respect to items in NP dictionary      확률계산
        for item in NP:
            # Assigns the probability of NP value by item key
            channel = NP[item]

            # Calculates trigram sentence probability / 이름은 수정할게 많아져서
            if len(sentence)-1 != index:
                bigram = calc.pow(calc.e, noisyChannelModel.ng.sentenceprobability(sentence[index-1]+item+sentence[index+1], 'bi', 'antilog'))
                
            else:
                bigram = calc.pow(calc.e, noisyChannelModel.ng.sentenceprobability(sentence[index-1]+item, 'bi', 'antilog'))

            # Assigns final probability into P dictionary
            P[item] =  channel * bigram * calc.pow(10,9)   #channel *

        # Sorts as parameters    확률 제일 높은거 저장
        P = sorted(P,key = P.get, reverse = True) #   lambda x: x[1]

        #print('bi*noise : ', P)
        if P == []:
            #correct = correct + word + ' '
            w =  noisyChannelModel.uniteWord(word)
            w = hgtk.text.compose(w)
            Res = 1
           #print("P가 없어 동사활용인지 검색 : ",w)
            if twitter.pos(w)[0][1] == 'Verb':
                for i in range(len(twitter.pos(w))):
                    if twitter.pos(w)[i][1]=="KoreanParticle":
                        print('word has particle')
                        correct = correct +w+ ' ' 
                        Res = 0
                        break
                if Res ==1 :
                    correct = correct +word+ ' ' 
                    #print('활용동사')
            else :
                P_can = list()
                for w in can_dict :
                    w =  noisyChannelModel.uniteWord(w)
                    w = hgtk.text.compose(w)
                    if len(hannanum.pos(w)) != 0 and hannanum.pos(w)[0][1] == 'P' :
                        for i in range(len(twitter.pos(w))):
                            if twitter.pos(w)[i][1]=="KoreanParticle":
                                Res = 0
                                break
                        if Res == 1 :
                            P_can.append(w)
                P_can = list(set(P_can))
                if P_can == [] :
                    correct = correct + word + ' '
                else :
                    correct = correct + P_can[0] + ' '
                #print("P_can :", P_can)

            
         #   P.append('')
        # Extracts the candidate which has highest probability
        else :
            correct = correct +P[0] + ' '       #correct : 단어 + 띄어쓰기
            
    # Prints the output response respect to input sentence iteratively
    correct = noisyChannelModel.uniteWord(correct)   # ㄲ,ㅒ 합치기
    correct = hgtk.text.compose(correct)             # 단어 합치기
   # print("correct : ", correct)
    correct = correct.split()
    #print("correct : " ,correct,"합 : ",correct + josa_word)      
    minimum = min(len(correct),len(josa_word))
    for i in range(0,minimum) :
        correct[i] = correct[i] + josa_word[i]
    print('Response: '," ".join(correct))
    
    gathering_sen = []
    gathering_sen.append(" ".join(correct))
    
    josa_word.clear()
# End program
for sen in gathering_sen:
    print(sen)

Loading Corpus from data file
Processing Corpus
Creating Unigram Model
Calculating Count for Unigram Model
Creating Trigram Model
Calculating Count for Trigram Model
Creating Bigram Model
Calculating Count for Bigram Model
['안녕하세요']
현재 검색 단어 : 안녕하세요
Response:  안녕하세요
['수험생', '여ㄹ러분']
현재 검색 단어 : 수험생
현재 검색 단어 : 여ㄹ러분
Response:  수험생 여러분
['어느새', '십험이', '왔네요']
현재 검색 단어 : 어느새
현재 검색 단어 : 십험
현재 검색 단어 : 왔네요
Response:  어느새 시험이 왔네요
['그동안', '고생', '많으ㅅ셨습니다']
현재 검색 단어 : 그동안
현재 검색 단어 : 고생
현재 검색 단어 : 많으ㅅ셨습니다
Response:  그동안 고생 많으셨습니다
그동안 고생 많으셨습니다
